# Process Raw Data to Clean Data

## All necessary Process

In [6]:
import os
import pandas as pd

def process_battery_data(input_file_path, output_file_path):
    # Load the Excel file
    xls = pd.ExcelFile(input_file_path)

    # Load the specific sheets into DataFrames
    qc_ex_df = pd.read_excel(xls, 'Qc_ex', index_col=0)
    voltage_df = pd.read_excel(xls, 'V_ex', index_col=0)
    qd_ex_df = pd.read_excel(xls, 'Qd_ex', index_col=0)

    # Create empty lists for new sheets
    qc_new_data = []
    voltage_new_data = []
    qd_ex_new_data = []

    # Process each row in Qc_ex to find the first maximum value position
    for index, row in qc_ex_df.iterrows():
        max_position = row.idxmax()  # Get the column name of the first maximum value
        max_index = row.index.get_loc(max_position)  # Get the positional index of the max value
        
        # Slice the row from the max position to the end and store in new DataFrame
        qc_new_data.append(row.iloc[max_index:].values)
        
        # Use the same position to slice the corresponding rows in voltage and qd_ex DataFrames
        voltage_new_data.append(voltage_df.loc[index].iloc[max_index:].values)
        qd_ex_new_data.append(qd_ex_df.loc[index].iloc[max_index:].values)

    # Convert lists to DataFrames
    qc_new_df = pd.DataFrame(qc_new_data, index=qc_ex_df.index)
    voltage_new_df = pd.DataFrame(voltage_new_data, index=qc_ex_df.index)
    qd_ex_new_df = pd.DataFrame(qd_ex_new_data, index=qc_ex_df.index)

    # Create a new Excel writer object and save the new DataFrames to it
    with pd.ExcelWriter(output_file_path) as writer:
        qc_new_df.to_excel(writer, sheet_name='Qc')
        voltage_new_df.to_excel(writer, sheet_name='voltage')
        qd_ex_new_df.to_excel(writer, sheet_name='Qd_ex')

def process_all_files_in_directory(input_directory, output_directory):
    # Ensure the output directory exists
    os.makedirs(output_directory, exist_ok=True)
    
    # Iterate over all files in the input directory
    for filename in os.listdir(input_directory):
        if filename.endswith('.xlsx'):
            input_file_path = os.path.join(input_directory, filename)
            output_file_name = os.path.splitext(filename)[0] + '_new.xlsx'
            output_file_path = os.path.join(output_directory, output_file_name)
            
            # Process each file
            process_battery_data(input_file_path, output_file_path)

# Function to process each file
def process_file(file_path, output_directory_path):
    xls = pd.ExcelFile(file_path)

    # Load the sheets into DataFrames
    df_voltage = pd.read_excel(xls, 'voltage')
    df_Qd_ex = pd.read_excel(xls, 'Qd_ex')

    # Removing the first unnamed column if it exists
    if df_voltage.columns[0].startswith('Unnamed'):
        df_voltage = df_voltage.iloc[:, 1:]

    if df_Qd_ex.columns[0].startswith('Unnamed'):
        df_Qd_ex = df_Qd_ex.iloc[:, 1:]

    # Function to filter and rearrange data based on voltage range
    def filter_and_rearrange(df_voltage, df_Qd_ex, min_voltage=2.19, max_voltage=2.401):
        filtered_voltage = []
        filtered_Qd_ex = []
        
        for i in range(df_voltage.shape[0]):
            voltage_row = df_voltage.iloc[i]
            valid_indices = voltage_row[(voltage_row >= min_voltage) & (voltage_row <= max_voltage)].index
            
            if not valid_indices.empty:
                first_valid_idx = valid_indices[0]
                last_valid_idx = valid_indices[-1]
                
                filtered_voltage.append(voltage_row[first_valid_idx:last_valid_idx + 1].reset_index(drop=True))
                filtered_Qd_ex.append(df_Qd_ex.iloc[i, first_valid_idx:last_valid_idx + 1].reset_index(drop=True))
            else:
                filtered_voltage.append(pd.Series())
                filtered_Qd_ex.append(pd.Series())
        
        filtered_voltage_df = pd.DataFrame(filtered_voltage).fillna('')
        filtered_Qd_ex_df = pd.DataFrame(filtered_Qd_ex).fillna('')
        
        return filtered_voltage_df, filtered_Qd_ex_df

    # Apply the function to filter and rearrange the data
    filtered_voltage_df, filtered_Qd_ex_df = filter_and_rearrange(df_voltage, df_Qd_ex)

    # # Delete all rows starting from the 201st row in both sheets
    # filtered_voltage_df = filtered_voltage_df.iloc[:200]
    # filtered_Qd_ex_df = filtered_Qd_ex_df.iloc[:200]

    # Construct the output file path
    output_file_path = os.path.join(output_directory_path, os.path.basename(file_path).replace('.xlsx', '_processed.xlsx'))

    # Save the processed DataFrames to new sheets in the same Excel file
    with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
        filtered_voltage_df.to_excel(writer, sheet_name='Filtered Voltage', index=False)
        filtered_Qd_ex_df.to_excel(writer, sheet_name='Filtered Qd_ex', index=False)

    # Reload the processed file for further extension
    xls = pd.ExcelFile(output_file_path)
    df_voltage = pd.read_excel(xls, 'Filtered Voltage')
    df_qd_ex = pd.read_excel(xls, 'Filtered Qd_ex')

    # Function to extend rows to maxi_columm
    def extend_row(row, maxi_columm):
        current_length = len(row.dropna())
        if current_length == 0:  # If the row is empty, return the row as is
            return row
        extended_row = pd.Series(row.dropna().values).reindex(range(maxi_columm))
        extended_row.interpolate(method='linear', inplace=True)
        extended_row.ffill(inplace=True)
        extended_row.bfill(inplace=True)
        return extended_row

    # Find the row with the maximum columns in Sheet voltage
    maxi_columm_voltage = df_voltage.notna().sum(axis=1).max()

    # Apply the function to each row in Sheet voltage
    df_voltage_extended = df_voltage.apply(lambda row: extend_row(row, maxi_columm_voltage), axis=1)

    # Find the row with the maximum columns in Sheet Qd_ex
    maxi_columm_qd_ex = df_qd_ex.notna().sum(axis=1).max()

    # Apply the function to each row in Sheet Qd_ex
    df_qd_ex_extended = df_qd_ex.apply(lambda row: extend_row(row, maxi_columm_qd_ex), axis=1)

    # Construct the final output file path
    final_output_file_path = os.path.join(output_directory_path, os.path.basename(file_path).replace('.xlsx', '_extended.xlsx'))
    
    # Save the modified DataFrames back to an Excel file
    with pd.ExcelWriter(final_output_file_path) as writer:
        df_voltage_extended.to_excel(writer, sheet_name='Voltage', index=False)
        df_qd_ex_extended.to_excel(writer, sheet_name='Qd', index=False)

    # print("The extended file has been saved to:", final_output_file_path)




In [7]:

def delete_columns_batch_2(input_folder, output_folder):
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(input_folder, filename)
            cleaned_file_path = os.path.join(output_folder, filename)

            # Load the Excel file
            excel_data = pd.ExcelFile(file_path)

            # Load the specific sheets 'voltage' and 'Qd_ex'
            voltage_df = pd.read_excel(file_path, sheet_name='voltage')
            qd_ex_df = pd.read_excel(file_path, sheet_name='Qd_ex')

            # Drop columns starting from the 374th to the end
            voltage_df_cleaned = voltage_df.iloc[:, :373]
            qd_ex_df_cleaned = qd_ex_df.iloc[:, :373]

            # Save the cleaned data back to a new Excel file
            with pd.ExcelWriter(cleaned_file_path, engine='openpyxl') as writer:
                voltage_df_cleaned.to_excel(writer, sheet_name='voltage', index=False)
                qd_ex_df_cleaned.to_excel(writer, sheet_name='Qd_ex', index=False)
                # Also write the original 'Qc' sheet
                excel_data.parse('Qc').to_excel(writer, sheet_name='Qc', index=False)

            # print(f"Processed and saved: {cleaned_file_path}")

    print("Processing complete.")

def delete_columns_batch_3_9(input_folder, output_folder):
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.xlsx'):
            file_path = os.path.join(input_folder, filename)
            cleaned_file_path = os.path.join(output_folder, filename)

            # Load the Excel file
            excel_data = pd.ExcelFile(file_path)

            # Load the specific sheets 'voltage' and 'Qd_ex'
            voltage_df = pd.read_excel(file_path, sheet_name='voltage')
            qd_ex_df = pd.read_excel(file_path, sheet_name='Qd_ex')

            # Drop columns starting from the 313th to the end
            voltage_df_cleaned = voltage_df.iloc[:, :315]
            qd_ex_df_cleaned = qd_ex_df.iloc[:, :315]

            # Save the cleaned data back to a new Excel file
            with pd.ExcelWriter(cleaned_file_path, engine='openpyxl') as writer:
                voltage_df_cleaned.to_excel(writer, sheet_name='voltage', index=False)
                qd_ex_df_cleaned.to_excel(writer, sheet_name='Qd_ex', index=False)
                # Also write the original 'Qc' sheet
                excel_data.parse('Qc').to_excel(writer, sheet_name='Qc', index=False)

            # print(f"Processed and saved: {cleaned_file_path}")

    print("Processing complete.")

def process_batch(input_directory_path, output_directory_path):
    for filename in os.listdir(input_directory_path):
        if filename.endswith('.xlsx') and not filename.endswith('_processed.xlsx') and not filename.endswith('_extended.xlsx'):
            file_path = os.path.join(input_directory_path, filename)
            process_file(file_path, output_directory_path)
    print(f"Folder {input_directory_path} processed successfully.")
    
def process_final_file(directory):
    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):  # Process only Excel files
            file_path = os.path.join(directory, filename)
            excel_file = pd.ExcelFile(file_path)

            # Initialize a dictionary to hold the modified dataframes
            sheets_dict = {}

            # Iterate through each sheet and modify it
            for sheet_name in excel_file.sheet_names:
                # Load the sheet into a DataFrame
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                # Drop columns from 130th to the end
                df = df.iloc[:, :362]
                # Save the modified DataFrame back to the dictionary
                sheets_dict[sheet_name] = df

            # Save the modified data back to the same Excel file
            with pd.ExcelWriter(file_path) as writer:
                for sheet_name, df in sheets_dict.items():
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    

def delete_processed_files(directory_path):
    # Get the list of files in the directory
    files_in_directory = os.listdir(directory_path)

    # Filter the files to find those ending with '_processed.xlsx'
    processed_files = [file for file in files_in_directory if file.endswith('_new_processed.xlsx')]

    # Delete the filtered files
    for file in processed_files:
        os.remove(os.path.join(directory_path, file))
        print(f"Deleted: {file}")


## Attract Voltage and Discharge Capacity

### Attract V,Qd,Qc

In [3]:
input_directory_1 = '.../Data_raw/b1'
output_directory_1 = '...//Data_raw/b1c'

input_directory_2 = '...//Data_raw/b2'
output_directory_2 ='...//Data_raw/b2c'

input_directory_3 = '...//Data_raw/b3'
output_directory_3 = '...//Data_raw/b3c'

input_directory_9 = '...//Data_raw/b9'
output_directory_9 = '...//Data_raw/b9c'

process_all_files_in_directory(input_directory_1, output_directory_1)
process_all_files_in_directory(input_directory_2, output_directory_2)
process_all_files_in_directory(input_directory_3, output_directory_3)
process_all_files_in_directory(input_directory_9, output_directory_9)



### Process Batch B1
This batch have some error data:
1. make it clean first 
2. Pick out Voltage and Qd
3. Delete all of the columm from 131th 

#### Rewrite

In [ ]:
# Directory containing the Excel files
directory = '...//Data_raw/b1c'

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(directory, filename)
        
        # Load the workbook
        wb = openpyxl.load_workbook(file_path)

        # Iterate through each sheet
        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            
            # Delete the first row after the header (assuming header is the first row)
            sheet.delete_rows(2)  # Deletes the second row (first row after the header)

        # Save the workbook (overwrites the original file)
        wb.save(file_path)


#### Generate to new path

##### Delete first row

In [8]:
import os
import openpyxl

# Directory containing the Excel files
input_directory = '...//Data_raw/b1c'
output_directory = '...//Data_raw/b1c_new'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Iterate through each file in the directory
for filename in os.listdir(input_directory):
    if filename.endswith('.xlsx'):
        file_path = os.path.join(input_directory, filename)
        
        # Load the workbook
        wb = openpyxl.load_workbook(file_path)

        # Iterate through each sheet
        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            
            # Delete the first row after the header (assuming header is the first row)
            sheet.delete_rows(2)  # Deletes the second row (first row after the header)

        # Save the workbook to the output directory
        output_file_path = os.path.join(output_directory, filename)
        wb.save(output_file_path)


##### Vmin < V < Vmax

In [3]:
# Define the directory containing the Excel files
input_directory_path = '...//Data_raw/b1c_new'
output_directory_path = '...//3_Range_22-24/b1c_new'

# Iterate over all Excel files in the directory and process them
for filename in os.listdir(input_directory_path):
    if filename.endswith('.xlsx') and not filename.endswith('_processed.xlsx') and not filename.endswith('_extended.xlsx'):
        file_path = os.path.join(input_directory_path, filename)
        process_file(file_path, output_directory_path)


##### Delete processed file

In [ ]:
b1c_new_path = '...//3_Range_22-24/b1c_new'
delete_processed_files(b1c_new_path)

##### Delete coluumn 

In [ ]:
# Define the directory containing the Excel files
directory = '...//3_Range_22-24/b1c_new'

# Iterate through each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):  # Process only Excel files
        file_path = os.path.join(directory, filename)
        excel_file = pd.ExcelFile(file_path)

        # Initialize a dictionary to hold the modified dataframes
        sheets_dict = {}

        # Iterate through each sheet and modify it
        for sheet_name in excel_file.sheet_names:
            # Load the sheet into a DataFrame
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            # Drop columns from 130th to the end
            df = df.iloc[:, :362]
            # Save the modified DataFrame back to the dictionary
            sheets_dict[sheet_name] = df

        # Save the modified data back to the same Excel file
        with pd.ExcelWriter(file_path) as writer:
            for sheet_name, df in sheets_dict.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Columns deleted and files saved successfully.")


### Process Batch 2, 3 and 9
1. The battery will be discharge to the end and then charge back again to 20% -> clean the charge back proces
2. Pick out Voltage and Qd
3. Delete all of the columm for charge back

##### Delete charge back

In [10]:
input_clean_folder_2 = '...//Data_raw/b2c/'
output_clean_folder_2 = '...//Data_raw/b2c_clean/'

input_clean_folder_3 = '...//Data_raw/b3c/'
output_clean_folder_3 = '...//Data_raw/b3c_clean/'

input_clean_folder_9 = '...//Data_raw/b9c/'
output_clean_folder_9 = '...//Data_raw/b9c_clean/'


delete_columns_batch_2(input_clean_folder_2, output_clean_folder_2)
delete_columns_batch_3_9(input_clean_folder_3, output_clean_folder_3)
delete_columns_batch_3_9(input_clean_folder_9, output_clean_folder_9)

Processing complete.
Processing complete.
Processing complete.


##### Vmin < V < Vmax

In [ ]:
# Define the directory containing the Excel files
input_directory_path_2 = '...//Data_raw/b2c_clean/'
output_directory_path_2 = '...//3_Range_22-24/b2c_new'

input_directory_path_3 = '...//Data_raw/b3c_clean/'
output_directory_path_3 = '...//3_Range_22-24/b3c_new'

input_directory_path_9 = '...//Data_raw/b9c_clean/'
output_directory_path_9 = '...//3_Range_22-24/b9c_new'

def process_batch(input_directory_path, output_directory_path):
    for filename in os.listdir(input_directory_path):
        if filename.endswith('.xlsx') and not filename.endswith('_processed.xlsx') and not filename.endswith('_extended.xlsx'):
            file_path = os.path.join(input_directory_path, filename)
            process_file(file_path, output_directory_path)
    print(f"Folder {input_directory_path} processed successfully.")
    
process_batch(input_directory_path_2, output_directory_path_2)
process_batch(input_directory_path_3, output_directory_path_3)
process_batch(input_directory_path_9, output_directory_path_9)

##### Delete processed file

In [ ]:
b2c_new_path = '...//3_Range_22-24/b2c_new'
delete_processed_files(b2c_new_path)

b3c_new_path = '...//3_Range_22-24/b3c_new'
delete_processed_files(b3c_new_path)

b9c_new_path = '...//3_Range_22-24/b9c_new'
delete_processed_files(b9c_new_path)

##### Delete coluumn

In [11]:
import pandas as pd
import os

# Define the directory containing the Excel files
directory_2 = '...//b2c_new'
directory_3 = '...//b3c_new'
directory_9 = '...//b9c_new'

process_final_file(directory_2)
process_final_file(directory_3)
process_final_file(directory_9)
